# Olympics Medals Data Wrangling

## Prepare data

In [3]:
# Libraries needed
import os
import zipfile
from pathlib import Path
import pandas as pd
import numpy as np

In [4]:
# Dataset downloader
def file_downloader():
    # Download Kaggle files using Kaggle API
    !kaggle datasets download heesoo37/120-years-of-olympic-history-athletes-and-results
    !kaggle datasets download piterfm/tokyo-2020-olympics


    # Create data directory
    try:
        os.mkdir('data')
    except FileExistsError:
        pass


    # Unzip files into data folder
    p = Path('.')

    for f in p.glob('*.zip'):
        with zipfile.ZipFile(f, 'r') as archive:
            archive.extractall(path=f'./data/{f.stem}')


    # Remove zip files
    files_to_remove = [file for file in os.listdir() if file.endswith('.zip')]

    for i in files_to_remove:
        os.remove(i)

file_downloader()


  0%|          | 0.00/5.43M [00:00<?, ?B/s]
 18%|█▊        | 1.00M/5.43M [00:00<00:00, 7.99MB/s]
100%|██████████| 5.43M/5.43M [00:00<00:00, 29.0MB/s]



  0%|          | 0.00/670k [00:00<?, ?B/s]
100%|██████████| 670k/670k [00:00<00:00, 5.96MB/s]
100%|██████████| 670k/670k [00:00<00:00, 5.91MB/s]


In [5]:
# Dataset loader
## Historic data
df = pd.read_csv('data/120-years-of-olympic-history-athletes-and-results/athlete_events.csv') 
regions = pd.read_csv('data/120-years-of-olympic-history-athletes-and-results/noc_regions.csv')
## 2021 data
athletes = pd.read_csv('data/tokyo-2020-olympics/athletes.csv')
coaches = pd.read_csv('data/tokyo-2020-olympics/coaches.csv')
medals = pd.read_csv('data/tokyo-2020-olympics/medals.csv')
medals_total = pd.read_csv('data/tokyo-2020-olympics/medals_total.csv')
officials = pd.read_csv('data/tokyo-2020-olympics/technical_officials.csv')

## Data cleaning

### Historic dataframe

In [6]:
## Historic dataframe
### Retrieve only observations that won a medal
df_clean = df[df['Medal'].notnull()]
### Fill out missing observations
df_clean = df_clean.fillna(0)
### Remove unnecessary columns
df_clean = df_clean[df_clean.columns[1:]]
df_clean = df_clean.drop('Weight', axis=1)
### Normalize height
df_clean['Height'] = df_clean['Height'] / 100
### Fix datatypes
df_clean['Age'] = df_clean['Age'].astype(int)

df_clean.head()

,Name,Sex,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,Edgar Lindenau Aabye,M,34,0.00,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
37,Arvo Ossian Aaltonen,M,30,0.00,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
38,Arvo Ossian Aaltonen,M,30,0.00,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
40,Juhamatti Tapio Aaltonen,M,28,1.84,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
41,Paavo Johannes Aaltonen,M,28,1.75,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze


### Medals dataframe

In [7]:
## Medals dataframe
### Create a copy
medals_clean = medals.copy()
### Convert date to datetime
medals_clean['Year'] = pd.to_datetime(medals['medal_date']).dt.year
### Get medal type
medals_clean['Medal'] = medals['medal_type'].apply(lambda x: x.split(' ')[0])
### Add column with the games edition and additional information
medals_clean['Games'] = '2021 Summer'
medals_clean['Season'] = 'Summer'
medals_clean['City'] = 'Tokyo'
### Rename some columns to match the historic dataframe
medals_clean['Sex'] = medals['athlete_sex']
medals_clean['Sport'] = medals['discipline']
medals_clean['NOC'] = medals['country_code']
medals_clean['Team'] = medals['country']
medals_clean['Event'] = medals['event']
medals_clean['Name'] = medals['athlete_name']
### Drop unnecessary columns
medals_clean = medals_clean[medals_clean.columns[-11:]]

medals_clean.head()

,Year,Medal,Games,Season,City,Sex,Sport,NOC,Team,Event,Name
0,2021,Gold,2021 Summer,Summer,Tokyo,X,Archery,KOR,Republic of Korea,Mixed Team,KIM Je Deok
1,2021,Gold,2021 Summer,Summer,Tokyo,X,Archery,KOR,Republic of Korea,Mixed Team,AN San
2,2021,Silver,2021 Summer,Summer,Tokyo,X,Archery,NED,Netherlands,Mixed Team,SCHLOESSER Gabriela
3,2021,Silver,2021 Summer,Summer,Tokyo,X,Archery,NED,Netherlands,Mixed Team,WIJLER Steve
4,2021,Bronze,2021 Summer,Summer,Tokyo,X,Archery,MEX,Mexico,Mixed Team,ALVAREZ Luis


### Athletes dataframe

In [8]:
## Athletes dataframe
### Create a copy
athletes_clean = athletes.copy()
### Get height in meters
athletes_clean['Height'] = athletes_clean[athletes_clean['height_m/ft'].notnull()]['height_m/ft'].apply(lambda x: x.split('/')[0])
### Fill missing values
athletes_clean = athletes_clean.fillna(0)
### Get age of athletes 
athletes_clean['Age'] = pd.to_datetime('2021-07-23') - pd.to_datetime(athletes_clean['birth_date'])
athletes_clean['Age'] = np.floor(athletes_clean['Age'] / np.timedelta64(1, 'Y')).astype('int')
### Normalize columns
athletes_clean['Name'] = athletes_clean['name']
athletes_clean['Country'] = athletes_clean['country']
### Fix datatypes
athletes_clean['Height'] = athletes_clean['Height'].astype(float)
### Remove unnecessary columns
athletes_clean = athletes_clean[athletes_clean.columns[-4:-1]]

athletes_clean.head()

,Height,Age,Name
0,0.00,26,AALERUD Katrine
1,1.65,28,ABAD Nestor
2,1.98,26,ABAGNALE Giovanni
3,2.00,25,ABALDE Alberto
4,1.92,32,ABALDE Tamara


### Merged dataframes

In [9]:
## Merge dataframes
medals_athletes = medals_clean.merge(athletes_clean, on='Name')
medals_athletes.head()

,Year,Medal,Games,Season,City,Sex,Sport,NOC,Team,Event,Name,Height,Age
0,2021,Gold,2021 Summer,Summer,Tokyo,X,Archery,KOR,Republic of Korea,Mixed Team,KIM Je Deok,0.0,17
1,2021,Gold,2021 Summer,Summer,Tokyo,M,Archery,KOR,Republic of Korea,Men's Team,KIM Je Deok,0.0,17
2,2021,Gold,2021 Summer,Summer,Tokyo,X,Archery,KOR,Republic of Korea,Mixed Team,AN San,0.0,20
3,2021,Gold,2021 Summer,Summer,Tokyo,W,Archery,KOR,Republic of Korea,Women's Team,AN San,0.0,20
4,2021,Gold,2021 Summer,Summer,Tokyo,W,Archery,KOR,Republic of Korea,Women's Individual,AN San,0.0,20


In [14]:
## Merge tokyo dataframe with historic
historic = df_clean.append(medals_athletes)
historic.head()

,Name,Sex,Age,Height,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,Edgar Lindenau Aabye,M,34,0.00,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
37,Arvo Ossian Aaltonen,M,30,0.00,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
38,Arvo Ossian Aaltonen,M,30,0.00,Finland,FIN,1920 Summer,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
40,Juhamatti Tapio Aaltonen,M,28,1.84,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
41,Paavo Johannes Aaltonen,M,28,1.75,Finland,FIN,1948 Summer,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze


## Save cleaned and combined dataframe

In [17]:
historic.to_csv('data/olympics.csv', index=False)